# Gurobi Exploration

In [1]:
#loading in libraries used, need to pip install gurobi, didn't add to docker yet
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import sys
import math
import random
from itertools import permutations
import configparser

In [2]:
config=configparser.ConfigParser()
config.read('../utils/config.ini')
cfg=config['gurobi api']

params = {
"WLSACCESSID": cfg['WLSACCESSID'],
"WLSSECRET": cfg['WLSSECRET'],
"LICENSEID": int(cfg['LICENSEID']),
}

env = gp.Env(params=params)

model_size_limited = gp.Model()
# Create the model within the Gurobi environment
model = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret


Set parameter LicenseID to value 2440575
Academic license 2440575 - for non-commercial use only - registered to je___@uchicago.edu
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2440575
Academic license 2440575 - for non-commercial use only - registered to je___@uchicago.edu


In [3]:
#loading data 

TruckPoints = pd.read_csv('../data/truck_service_pts_galv.csv')

In [4]:
#splitting by indoor outdoor
DropoffOnly = TruckPoints.copy()

first_row = DropoffOnly.iloc[0:1, :]
DropoffOnly = DropoffOnly[DropoffOnly['Weekly_Dropoff_Totes'] >= 1]

# Concatenate the first row with the filtered DataFrame
DropoffOnly = pd.concat([first_row, DropoffOnly])

len(DropoffOnly)

171

In [5]:
#Drop off Only Route, changing Lat and Long columns to Coordinate for Gurobi 

def create_coordinates(row):
    return f'({row["Latitude"]}, {row["Longitude"]})'

# Apply the function to create a new 'coordinates' column
DropoffOnly['Coordinates'] = DropoffOnly.apply(create_coordinates, axis=1)

In [13]:
#Starting Model

#number of locations
n=171
locations = [*range(n)]

#number of vans 
K= 2
vans = [*range(K)]

#number of points is coordinates
points = DropoffOnly['Coordinates'].tolist()

# Extract coordinates from strings and convert to floats
points = [[float(coord) for coord in point.strip('()').split(',')] for point in points]

# Dict of Euclidean Distance
time = {(i, j): math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
        for i in locations for j in locations if i != j}

In [22]:
print(time)

{(0, 1): 0.022052503097151628, (0, 2): 0.06596447726807259, (0, 3): 0.04695881283220201, (0, 4): 0.1119836636609162, (0, 5): 0.07171993717927751, (0, 6): 0.053965092839079465, (0, 7): 0.06895620855796394, (0, 8): 0.06863558094312716, (0, 9): 0.06817398044005077, (0, 10): 0.06610255371526554, (0, 11): 0.06620184072402806, (0, 12): 0.06594506669675479, (0, 13): 0.06494260490248545, (0, 14): 0.06159968802258784, (0, 15): 0.0597604589454404, (0, 16): 0.04504132316551604, (0, 17): 0.014741455890449031, (0, 18): 0.018549652287849556, (0, 19): 0.04584481219287407, (0, 20): 0.06596447726807259, (0, 21): 0.07415024094162072, (0, 22): 0.02092732188503592, (0, 23): 0.06141843892619416, (0, 24): 0.06090650031260995, (0, 25): 0.04817827985980383, (0, 26): 0.0644035245103849, (0, 27): 0.06661175224095253, (0, 28): 0.07696905030205238, (0, 29): 0.07678913342192366, (0, 30): 0.07678550123721174, (0, 31): 0.018074140034865745, (0, 32): 0.06491212709478697, (0, 33): 0.04175979233976248, (0, 34): 0.07188

In [23]:
#adding variables from gurobi, these can be changed:

# x =1, if van k  visits and goes directly from location  i to location j 
x = model.addVars(time.keys(), vans, vtype=GRB.BINARY, name='FromToBy')

# y = 1, if customer i is visited by van k
y = model.addVars(locations, vans, vtype=GRB.BINARY, name='visitBy')

# Number of vans used is a decision variable
z = model.addVars(vans, vtype=GRB.BINARY, name='used')

# Travel time per van, what is ub
t = model.addVars(vans, ub=240, name='travelTime')

#travel time
s = model.addVar(name='maxTravelTime')

In [29]:
#adding contraits from gurobi, removing time constraint for now

# Travel time constraint
# Exclude the time to return to the depot

visitCustomer = model.addConstrs((y[i,k] <= z[k]  for k in vans for i in locations if i > 0), name='visitCustomer' )


travelTime = model.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in vans), 
                          name='travelTimeConstr' )


# Visit all customers
visitAll = model.addConstrs((y.sum(i,'*') == 1 for i in locations if i > 0), name='visitAll')

# Depot constraint
depotConstr = model.addConstrs((y[0, k] == z[k] for k in vans), name='depotConstr')

# Arriving at a customer location constraint
ArriveConstr = model.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr')

# Leaving a customer location constraint
LeaveConstr = model.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr')

breakSymm = model.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in vans if k>0), name='breakSymm')

maxTravelTime = model.addConstrs((t[k] <= s for k in vans), name='maxTravelTimeConstr')

In [30]:
model.ModelSense = GRB.MINIMIZE
model.setObjectiveN(z.sum(), 0, priority=1, name="Number of vans")
model.setObjectiveN(s, 1, priority=0, name="Travel time")

In [31]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour not containing depot (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [32]:
model.write('weekly_tote_distribution.rlp')

# Run optimization engine
model._x = x
model.Params.LazyConstraints = 1
model.optimize(subtourelim)

# m.dispose()
# env.dispose()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2440575 - for non-commercial use only - registered to je___@uchicago.edu
Optimize a model with 6344 rows, 233947 columns and 969223 nonzeros
Model fingerprint: 0x126d0c38
Variable types: 11 continuous, 233936 integer (233936 binary)
Coefficient statistics:
  Matrix range     [2e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+02]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 1892 rows and 178 columns
Presolve time

In [34]:
# Print optimal routes
for k in vans:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"Route for van {k}: {i}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            print(f" -> {i}", end='')
            if i == 0:
                break
        print(f". Travel time: {round(t[k].X,2)} min")

print(f"Max travel time: {round(s.X,2)}")



Route for van 0: 0 -> 117 -> 125 -> 141 -> 139 -> 123 -> 17 -> 18 -> 140 -> 57 -> 118 -> 116 -> 168 -> 166 -> 167 -> 35 -> 22 -> 152 -> 146 -> 114 -> 108 -> 162 -> 63 -> 6 -> 94 -> 97 -> 3 -> 41 -> 42 -> 98 -> 44 -> 90 -> 82 -> 24 -> 77 -> 23 -> 14 -> 26 -> 71 -> 110 -> 66 -> 68 -> 89 -> 150 -> 62 -> 59 -> 11 -> 10 -> 65 -> 155 -> 70 -> 69 -> 154 -> 67 -> 12 -> 64 -> 153 -> 142 -> 61 -> 53 -> 147 -> 148 -> 50 -> 49 -> 36 -> 47 -> 34 -> 5 -> 157 -> 101 -> 48 -> 7 -> 105 -> 104 -> 102 -> 8 -> 54 -> 55 -> 56 -> 58 -> 9 -> 149 -> 27 -> 165 -> 13 -> 169 -> 76 -> 72 -> 32 -> 2 -> 20 -> 38 -> 21 -> 30 -> 29 -> 28 -> 144 -> 103 -> 120 -> 109 -> 121 -> 4 -> 160 -> 164 -> 119 -> 129 -> 37 -> 40 -> 43 -> 52 -> 51 -> 46 -> 73 -> 75 -> 74 -> 15 -> 156 -> 79 -> 80 -> 81 -> 85 -> 87 -> 158 -> 86 -> 84 -> 83 -> 88 -> 91 -> 92 -> 93 -> 95 -> 96 -> 25 -> 99 -> 19 -> 16 -> 133 -> 100 -> 106 -> 161 -> 163 -> 107 -> 130 -> 134 -> 111 -> 132 -> 112 -> 136 -> 137 -> 113 -> 138 -> 78 -> 115 -> 31 -> 127 -> 12